This notebook contains some useful functions for computing the *tf-idf* for the data with arXiv abstracts.

Note here the use of specific pandas function like *concat*, *value_counts* and *groupby* which make possible to speed up the computations.

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
data = pd.read_csv("./arxiv_articles.csv", sep="|")

In [3]:
data.loc[0:5, :]

,id,title,authors,arxiv_primary_category,summary,published,updated
0,http://arxiv.org/abs/2001.05867v1,$σ$-Lacunary actions of Polish groups,Jan Grebik,math.LO,We show that every essentially countable orbit...,2020-01-16T15:09:02Z,2020-01-16T15:09:02Z
1,http://arxiv.org/abs/1303.6933v1,Hans Grauert (1930-2011),Alan Huckleberry,math.HO,Hans Grauert died in September of 2011. This a...,2013-03-27T19:23:57Z,2013-03-27T19:23:57Z
2,http://arxiv.org/abs/1407.3775v1,A New Proof of Stirling's Formula,Thorsten Neuschel,math.HO,A new simple proof of Stirling's formula via t...,2014-07-10T11:26:39Z,2014-07-10T11:26:39Z
3,http://arxiv.org/abs/math/0307381v3,On Dequantization of Fedosov's Deformation Qua...,Alexander V. Karabegov,math.QA,To each natural deformation quantization on a ...,2003-07-30T06:20:33Z,2003-09-20T01:29:18Z
4,http://arxiv.org/abs/1604.06794v1,Cyclic extensions are radical,Mariano Suárez-Álvarez,math.HO,We show that finite Galois extensions with cyc...,2016-04-21T22:24:54Z,2016-04-21T22:24:54Z
5,http://arxiv.org/abs/1712.09576v2,The Second Main Theorem in the hyperbolic case,Min Ru;Nessim Sibony,math.CV,We develop Nevanlinna's theory for a class of ...,2017-12-27T13:17:08Z,2019-01-03T07:51:11Z


In [4]:
def naif_regex_tokenize(text):
    """
    This is a very naif way of tokenize a text. Just using the
    regular expression "[a-z]" that will match any single word
    in lowercase.
    Returns a list with all the tokens.
    """
    p = re.compile("[a-z]+")
    return p.findall(text.lower())

def compute_tf(d):
    """
    Compute the tf for a given document d.
    The formula used is 
    
        tf(t, d) = 0.5 + 0.5 * (count(t, d)/max(count(t',d) for t' in d))
    
    This prevents bias in longer documents.
    """
    terms = pd.Series(naif_regex_tokenize(d))
    term_counts = terms.value_counts()
    max_tc = max(term_counts)
    return 0.5 + 0.5 * (term_counts / max_tc)

def compute_idf(D):
    """
    The input D is a list of pandas.Series
    having as each element, the term frequency 
    computed by the function compute_tf.
    """
    N = len(D)
    all_terms = pd.concat(D)
    nt = all_terms.index.value_counts() # The number of documents containing the term "t"
    return np.log(N / nt)

def compute_tf_idf_document(tf_document, idf):
    """Compute the tf-idf for each term in a document of the corpus

    Keyword arguments:
    tf_document -- list with the frequency of each term inside the document
    idf -- the idf value for each term in the corpus
    """
    return tf_document * np.array([idf[i] for i in tf_document.index])
    
def compute_tf_idf_corpus(D):
    """Compute the tf-idf for each term in a corpus

    Keyword arguments:
    D -- pandas Series containing a collection of documents in text format
    
    returns
        list of pandas Series containing the tf-idf(t, d, D) for each term
        inside each document of the corpus D
    """
    term_freq = [compute_tf(d) for d in D]
    idf = compute_idf(term_freq)
    return [compute_tf_idf_document(d, idf) for d in term_freq]


In [6]:
s = data['summary'][0]
print(s)

We show that every essentially countable orbit equivalence relation induced by a continuous action of a Polish group on a Polish space is $\sigma$-lacunary. In combination with [Invent. Math.201 (1), 309-383, 2015] we obtain a straightforward proof of the result from [Adv. Math.307, 312-343,2017] that every essentially countable equivalence relation that is induced by an action of abelian non-archimedean Polish group is essentially hyperfinite.


In [9]:
D1 = data['summary'][:10]

In [12]:
tf_idf = compute_tf_idf_corpus(data.loc[:, "summary"]) # Sauvegarder avec pickle

At this stage we have the tf-idf values at each document. In order to select the *most important* terms (i.e. the terms with higher tf-idf values), we compute the **mean** of the tf-idf for each term.

In [18]:
tf_idf[1]

his                4.659259
in                 0.121669
hans               7.447268
grauert            7.967129
of                 0.018285
accomplishments    6.507696
died               5.887687
detail             3.147262
september          4.729013
life               2.903803
some               1.532848
reviews            4.234578
and                0.061446
this               0.350511
recalls            6.319210
major              2.690017
mathematics        3.935670
article            2.477727
dtype: float64

In [19]:
print(data["summary"][1])

Hans Grauert died in September of 2011. This article reviews his life in mathematics and recalls some detail his major accomplishments.


In [13]:
all_terms = pd.concat(tf_idf)

In [45]:
mean_tf_idf = all_terms.groupby(all_terms.index).mean()

In [51]:
sorted_tf_idf = mean_tf_idf.sort_values(ascending=False)

In [55]:
sorted_tf_idf[:1000]

karatsuba             10.622838
erdi                  10.622838
sagetex               10.622838
bioelectrodynamics    10.622838
koras                 10.622838
                        ...    
verschiebung           7.967129
verses                 7.967129
carlisle               7.967129
cotter                 7.967129
cotranslational        7.967129
Length: 1000, dtype: float64

In [56]:
all_terms["cotter"]

7.967128798944532

In [57]:
unique["cotter"]

1


   "the" "a" "model"
d1    0.1 5.8 9
d2    4.7 1.0 3
d3    8.0 2.4 6.0
d4    0.3 9.1 3.2

k = 2
groupe 1    groupe 2
(d1, d4, d3)  (d2)

K = 5

(d1, d50)     (d2, d3, d10)      (d9, d8)     (d6, d7)    (d20, d32, ...)




x = [0.1, 5.8, 9]
norme2(x) = S

S = sqrt(0.1**2 + 5.8**2 + 9**2)
x_normalise = [0.1, 5.8, 9] / S
norme2(x_normalise) = 1


## Maintenant c'est à vous

Utiliser le code et les fonctions ci-dessus pour :

1. Proposer un dictionnaire des termes (une liste de mots qui peuvent être "informatives"). Par exemple, un mot qui est présent dans plus de 10 documents différents et qui a un fort valeur de *tf-idf* peut etre "informatif". 

2. Utiliser ces termes pour calculer, pour chaque document, un vecteur *tf-idf*. Ce vecteur aura les valeurs du tf-idf de chaqu'un des termes. Par example, si la liste de termes est ["float", "genetic", "circular"] et qu'on a 4 document. On doit produire une matrice de 4 lignes et 3 colonnes :
```
0.1 5.8 9
4.7 1.0 3
8.0 2.4 6.0
0.3 9.1 3.2
```   
Ici, chaque ligne contient les valeurs de *tf-idf* de ["float", "genetic", "circular"] (dans cet ordre).

3. Normaliser les lignes de cette matrice. La norme 2 des vecteurs *tf-idf* représentés à chaque ligne doit être 1. Les étapes **2** et **3** font partie de ce qu'on appelle *feature extraction*. 

4. Executer l'exemple décrit [ici](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction). Appliquer la même analyse sur notre jeu des données.

5. Faire une implémentation de l'algorithme k-means. Voir les liens : https://en.wikipedia.org/wiki/K-means_clustering, https://en.wikipedia.org/wiki/K-means%2B%2B, https://fr.wikipedia.org/wiki/K-moyennes

6. Implémenter une fonction permettant de trier les documents en fonction du résultat de l'algorithme k-means avec **k** groupes.

7. Executer l'exemple décrit [ici](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py) puis appliquer la même analyse sur notre jeu des données

